# Tagger 

This example shows the definition of a *tagger model* using simple `HashEmbed` and `Relu` layers.


In [1]:
from thinc.api import prefer_gpu
prefer_gpu()

from thinc.api import fix_random_seed
fix_random_seed(0)

from tqdm import tqdm



In [2]:
def train_model(data, model, optimizer, n_iter, batch_size):
    (train_X, train_y), (test_X, test_y) = data
    model.initialize(X=train_X[:5], Y=train_y[:5])
    for n in range(n_iter):
        loss = 0.0
        batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            d_loss = []
            for i in range(len(Yh)):
                d_loss.append(Yh[i] - Y[i])
                loss += ((Yh[i] - Y[i]) ** 2).sum()
            backprop(d_loss)
            model.finish_update(optimizer)
        score = evaluate(model, test_X, test_y, batch_size)
        print(f"{n}\t{loss:.2f}\t{score:.3f}")
        
def evaluate(model, test_X, test_Y, batch_size):
    correct = 0
    total = 0
    for X, Y in model.ops.multibatch(batch_size, test_X, test_Y):
        Yh = model.predict(X)
        for yh, y in zip(Yh, Y):
            correct += (y.argmax(axis=1) == yh.argmax(axis=1)).sum()
            total += y.shape[0]
    return float(correct / total)

In [3]:
from thinc.api import HashEmbed, Maxout, Softmax, expand_window, Relu
from thinc.api import residual, strings2arrays, with_array, clone, chain, concatenate


width = 32
n_tags = 17
vector_width = 16

model = chain(
    strings2arrays(),
    with_array(
        chain(
            HashEmbed(width, vector_width, column=0),
            expand_window(1),
            Relu(nI=96, nO=width),
            Relu(nI=width, nO=width),
            Softmax(nI=width, nO=n_tags)
        )
    )
)


In [4]:
import ml_datasets


CONFIG = """
[hyper_params]
width = 32
vector_width = 16
learn_rate = 0.001

[training]
n_iter = 10
batch_size = 128

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""


from thinc.api import registry, Config

config = Config().from_str(CONFIG)
loaded_config = registry.make_from_config(config)

optimizer = loaded_config["optimizer"]
n_iter = loaded_config["training"]["n_iter"]
batch_size = loaded_config["training"]["batch_size"]

(train_X, train_y), (test_X, test_y) = data = ml_datasets.ud_ancora_pos_tags(limit=1000)



In [5]:
model.initialize(X=train_X[:5], Y=train_y[:5])
train_model(data, model, optimizer, n_iter, batch_size)

  0%|          | 0/8 [00:00<?, ?it/s]2	30983.96	0.201
3	30440.91	0.181
  0%|          | 0/8 [00:00<?, ?it/s]4	29765.40	0.180
5	29511.00	0.180
  0%|          | 0/8 [00:00<?, ?it/s]6	29391.66	0.180
7	29217.76	0.180
  0%|          | 0/8 [00:00<?, ?it/s]8	28961.60	0.243
9	28540.47	0.371
